# BioMedCLIP 의료 영상-텍스트 매칭

> **Purpose:** BioMedCLIP 모델을 활용하여 의료 영상과 증상 텍스트 간의 유사도를 분석합니다.
>
> **모델:** `microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224`
> - 학습 데이터: PMC-15M (1,500만 바이오메디컬 이미지-텍스트 쌍)
> - 역할: 영상-증상 cross-modal retrieval + 위급도 평가
> - 결과 저장: `P2T2.ai_results.biomedclip_results`
>
> **카탈로그:** `P2T2`

## 0. 카탈로그 설정

In [ ]:
spark.sql("USE CATALOG P2T2")
print("✅ Catalog: P2T2")

## 1. 라이브러리 및 모델 로드

In [ ]:

import torch
import numpy as np
from pyspark.sql import functions as F

# BioMedCLIP 모델 로드
# from open_clip import create_model_and_transforms, get_tokenizer
# model, preprocess_train, preprocess_val = create_model_and_transforms(
#     'hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224'
# )
# tokenizer = get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')

print("🔧 BioMedCLIP 모델 설정 준비")
print("   → microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224")

## 2. 영상-텍스트 유사도 계산

In [ ]:

def compute_similarity(image_features, text_features):
    """
    코사인 유사도 계산 (0.0 ~ 1.0)
    """
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    similarity = (image_features @ text_features.T).item()
    return max(0.0, min(1.0, similarity))

print("✅ 유사도 계산 함수 준비 완료")

## 3. Context Fusion (바이탈 + 영상)

In [ ]:

def context_fusion(vital_data, image_analysis_result):
    """
    BioMedCLIP이 바이탈 데이터와 영상 분석 결과를 결합하여 위급도를 평가합니다.
    
    실제 컬럼: avg_heart_rate, avg_systolic_bp, avg_diastolic_bp, avg_spo2,
              avg_temperature, avg_respiratory_rate, max_risk_score
    """
    # 텍스트 프롬프트 구성
    clinical_text = f"""
    Patient vitals: HR {vital_data.get('avg_heart_rate', 'N/A')} bpm, 
    BP {vital_data.get('avg_systolic_bp', 'N/A')}/{vital_data.get('avg_diastolic_bp', 'N/A')} mmHg,
    SpO2 {vital_data.get('avg_spo2', 'N/A')}%,
    Temp {vital_data.get('avg_temperature', 'N/A')}°C,
    RR {vital_data.get('avg_respiratory_rate', 'N/A')}/min.
    Risk score: {vital_data.get('max_risk_score', 'N/A')}/1.0.
    Imaging findings: {image_analysis_result.get('findings', 'No findings available')}.
    """
    
    # 후보 진단 텍스트 목록
    candidate_diagnoses = [
        "Acute ischemic stroke with middle cerebral artery occlusion",
        "Pulmonary embolism with right ventricular strain",
        "Acute myocardial infarction with ST elevation",
        "Pneumothorax with mediastinal shift",
        "Normal findings, no acute pathology",
    ]
    
    # 유사도 계산 (시뮬레이션 — 실제: BioMedCLIP 임베딩)
    results = []
    for diagnosis in candidate_diagnoses:
        similarity = np.random.uniform(0.3, 0.95)
        results.append({"diagnosis": diagnosis, "similarity": round(similarity, 4)})
    
    results.sort(key=lambda x: x["similarity"], reverse=True)
    
    # 위급도 평가
    top_sim = results[0]["similarity"]
    urgency = "CRITICAL" if top_sim > 0.8 else "WARNING" if top_sim > 0.5 else "STABLE"
    
    return {
        "patient_id": vital_data.get("patient_id"),
        "top_diagnosis": results[0]["diagnosis"],
        "top_similarity": str(results[0]["similarity"]),
        "urgency_level": urgency,
    }

print("✅ Context Fusion 함수 준비 완료")

## 4. 배치 매칭 및 결과 저장

In [ ]:

def batch_biomedclip_matching():
    """
    Gold Layer 환자 데이터와 DICOM 영상 매칭 후 P2T2.ai_results에 저장합니다.
    """
    df_patients = spark.table("P2T2.gold.patient_clinical_summary").collect()
    df_dicom = spark.table("P2T2.silver.cleaned_dicom_metadata").collect()
    
    results = []
    for p in df_patients:
        vital_data = {k: str(v) if v is not None else "N/A" for k, v in p.asDict().items()}
        d = next((d for d in df_dicom if d["patient_id"] == p["patient_id"]), None)
        image_result = {"findings": d["finding_labels"] if d else "No DICOM available"}
        
        result = context_fusion(vital_data, image_result)
        results.append(result)
    
    if results:
        df_results = spark.createDataFrame(results)
        df_results = df_results.withColumn("created_at", F.current_timestamp())
        df_results.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable("P2T2.ai_results.biomedclip_results")
        print(f"✅ BioMedCLIP 매칭 저장: {len(results)}건 → P2T2.ai_results.biomedclip_results")
    else:
        print("⚠️ 매칭할 환자 데이터 없음")

batch_biomedclip_matching()